In [1]:
import os
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf

#### TO suppress
import sys
sys.path.insert(0,'../debvader/')
#### 
import train
import debvader
#from debvader import train
#from debvader.debvader import load_deblender, deblend ### TO UNCOMMENT

/pbs/throng/lsst/users/barcelin/debvader/debvader


### Download and format data

Download dataset that will be used for training. It has been generated using the code in https://github.com/BastienArcelin/dc2_img_generation and the stamps are generated using the XXX function see notebook XXX. The size of the stamp is then fixed to 59x59 pixels.

In [2]:
path_folder = os.path.dirname(os.path.abspath("__file__"))
images = np.load(path_folder+'/../data/dc2_imgs/imgs_dc2.npy', mmap_mode = 'c')

In [3]:
images.shape

(10, 59, 59, 6)

Separate training data and labels to feed the VAE and deblender. You should be careful that the number of filter in the data for trainng correspond to the number of bands considered for the network (default is six).

In [4]:
training_data_vae = np.array((images[:5], images[:5]))
validation_data_vae = np.array((images[5:], images[5:]))

training_data_deblender = np.array((images[:5], images[:5]))
validation_data_deblender = np.array((images[5:], images[5:]))

### Train the VAE and deblender with architecture from the debvader paper (https://arxiv.org/abs/2005.12039)

In [5]:
hist_vae, hist_deblender, net = train.train_deblender("lsst",
                                                      from_survey = "dc2", 
                                                      epochs = 2, 
                                                      training_data_vae = training_data_vae, 
                                                      validation_data_vae = validation_data_vae, 
                                                      training_data_deblender = training_data_deblender, 
                                                      validation_data_deblender = validation_data_deblender)

in cropping
6
Instructions for updating:
Do not pass `graph_parents`.  They will  no longer be used.
VAE model
Model: "model_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         [(None, 59, 59, 6)]       0         
_________________________________________________________________
model (Model)                (None, 560)               3741224   
_________________________________________________________________
multivariate_normal_tri_l (M ((None, 32), (None, 32))  0         
_________________________________________________________________
model_1 (Model)              (None, 59, 59, 6)         4577228   
Total params: 8,318,452
Trainable params: 8,318,440
Non-trainable params: 12
_________________________________________________________________

Start the training
Train on 5 samples, validate on 5 samples
Epoch 1/2

Epoch 00001: mse improved from inf to 0.11485, saving model to

### Dataset with a different number of filters

The number of available filters is different for each survey. For example, five filters are available for DES data. To change that you should specify it in the train_deblender function, as well as if the channels appear last or first in the data array.

In [6]:
training_data_vae.shape

(2, 5, 59, 59, 6)

Here the channels appear last in the data array. So the channel_first option in the train_deblender function is set as False (default setting).

In [7]:
hist_vae, hist_deblender, net = train.train_deblender("des",
                                                      from_survey = "dc2", 
                                                      epochs = 2, 
                                                      training_data_vae = training_data_vae, 
                                                      validation_data_vae = validation_data_vae, 
                                                      training_data_deblender = training_data_deblender, 
                                                      validation_data_deblender = validation_data_deblender,
                                                      nb_of_bands = 5,
                                                      channel_last = True)

in cropping
5
VAE model
Model: "model_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_6 (InputLayer)         [(None, 59, 59, 5)]       0         
_________________________________________________________________
model_3 (Model)              (None, 560)               3740932   
_________________________________________________________________
multivariate_normal_tri_l_1  ((None, 32), (None, 32))  0         
_________________________________________________________________
model_4 (Model)              (None, 59, 59, 5)         4576650   
Total params: 8,317,582
Trainable params: 8,317,572
Non-trainable params: 10
_________________________________________________________________
6 5
The number of bands in the data does not correspond to the number of filters in the network. Correct this before starting again.


ValueError: 

We need to change the training data format. For example just train on the five fist channels.

In [8]:
training_data_vae_deslike = np.array((images[:5,:,:,:5], images[:5,:,:,:5]))
validation_data_vae_deslike = np.array((images[5:,:,:,:5], images[5:,:,:,:5]))

training_data_deblender_deslike = np.array((images[:5,:,:,:5], images[:5,:,:,:5]))
validation_data_deblender_deslike = np.array((images[5:,:,:,:5], images[5:,:,:,:5]))

Now try the training:

In [10]:
path_output = '../data/weights/dc2/not_normalised/'
latest = tf.train.latest_checkpoint(path_output)

Warning: we cannot use the weights from the network trained on DC2 as the data has six channels in that case. We need to set the from_survey option to None.

In [12]:
hist_vae, hist_deblender, net = train.train_deblender("des",
                                                      from_survey = None, 
                                                      epochs = 2, 
                                                      training_data_vae = training_data_vae_deslike, 
                                                      validation_data_vae = validation_data_vae_deslike, 
                                                      training_data_deblender = training_data_deblender_deslike, 
                                                      validation_data_deblender = validation_data_deblender_deslike,
                                                      nb_of_bands = 5,
                                                      channel_last = True)

in cropping
5
VAE model
Model: "model_11"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_12 (InputLayer)        [(None, 59, 59, 5)]       0         
_________________________________________________________________
model_9 (Model)              (None, 560)               3740932   
_________________________________________________________________
multivariate_normal_tri_l_3  ((None, 32), (None, 32))  0         
_________________________________________________________________
model_10 (Model)             (None, 59, 59, 5)         4576650   
Total params: 8,317,582
Trainable params: 8,317,572
Non-trainable params: 10
_________________________________________________________________

Start the training
Train on 5 samples, validate on 5 samples
Epoch 1/2

Epoch 00001: mse improved from inf to 0.06085, saving model to ../data/weights/des/vae/mse/weights_noisy_v4.ckpt

Epoch 00001: loss improved from in